In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import re
import json
import numpy as np
import pandas as pd
from unidecode import unidecode
from selenium.webdriver.chrome.options import Options


In [2]:
def get_links():
    driver = webdriver.Chrome()
    driver.set_window_size(900,800)
    elements = []
    links = []
    driver.get("https://torob.com/browse/99/%D9%84%D9%BE-%D8%AA%D8%A7%D9%BE-%D9%88-%D9%86%D9%88%D8%AA-%D8%A8%D9%88%DA%A9-laptop/?stock_status=new")
    SCROLL_PAUSE_TIME = 0.5
    ctr = 0
    last_height = driver.execute_script("return document.body.scrollHeight")
    while (True):
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(SCROLL_PAUSE_TIME)
        driver.execute_script("window.scrollTo(0,500)")
        time.sleep(SCROLL_PAUSE_TIME)
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(SCROLL_PAUSE_TIME)
        elements = driver.find_elements(By.CSS_SELECTOR,".jsx-2514672dc9197d80 a")
        if (driver.execute_script("return document.body.scrollHeight") != last_height):
            last_height = driver.execute_script("return document.body.scrollHeight")
            ctr = 0
        else:
            if (ctr == 5):
                break
            ctr+=1
    links = [elem.get_attribute("href") for elem in elements]
    driver.quit()
    return links

In [3]:
links = get_links()
print(len(links))
print(links[len(links)-1])

1176
https://torob.com/p/8791f27d-a9fb-4771-9f04-df3c954cb8d3/hp-omen-16-r7-16gb-512ssd-8gb-rx-6600m/


In [52]:
def get_json_data(links):
    options = Options()
    options.add_argument('--headless=new')
    data = []
    company_name = []
    company_name2 = []
    price = []
    ct = 0
    for link in links:
        base = "https://api.torob.com/v4/base-product/details-log-click/?prk="
        id = re.findall("[a-z0-9]{8}-[a-z0-9]{4}-[a-z0-9]{4}-[a-z0-9]{4}-[a-z0-9]{12}",link)
        base += id[0]
        driver = webdriver.Chrome(options=options)
        driver.get(base);
        raw_obj = json.loads(driver.find_element(By.TAG_NAME,"pre").text)
        data.append(raw_obj["attributes"])
        price.append(raw_obj["price"])
        company_name.append(raw_obj["name1"])
        company_name2.append(raw_obj["name2"])
    driver.quit()
    return data,company_name,company_name2,price

In [53]:
data,company_name,company_name2,price = get_json_data(links)
print(data)
print(company_name)
print(company_name2)
print(price)

[{'stock_status': 'new', 'cpu': 'Celeron', 'screen_size': '15.6 inch', 'ram': '4 gb', 'graphic_ram': 'unified', 'hdd': '1 tb'}, {'cpu': 'Core i7', 'ram': '16 gb', 'ssd': '512 gb', 'graphic_ram': '8 gb', 'screen_size': '15.6 inch', 'stock_status': 'new'}, {'cpu': 'M1', 'ram': '8 gb', 'ssd': '256 gb', 'graphic_ram': 'unified', 'screen_size': '13 inch', 'stock_status': 'new'}, {'cpu': 'Core i5', 'ram': '16 gb', 'ssd': '512 gb', 'graphic_ram': '4 gb', 'screen_size': '15.6 inch', 'stock_status': 'new'}, {'cpu': 'Core i3', 'hdd': '1 tb', 'ram': '12 gb', 'ssd': '256 gb', 'graphic_ram': 'unified', 'screen_size': '15.6 inch', 'stock_status': 'new'}, {'cpu': 'Core i5', 'ram': '8 gb', 'ssd': '512 gb', 'graphic_ram': '4 gb', 'screen_size': '15.6 inch', 'stock_status': 'new'}, {'cpu': 'Core i5', 'ram': '8 gb', 'ssd': '512 gb', 'graphic_ram': '4 gb', 'screen_size': '15.6 inch', 'stock_status': 'new'}, {'cpu': 'Core i5', 'ram': '8 gb', 'ssd': '512 gb', 'graphic_ram': '4 gb', 'screen_size': '15.6 inch

In [103]:
#the part bellow should be executed if you have the dataset

In [104]:
def create_number(inp):
    if (isinstance(inp,str)):
        if ('tb' in inp):
            inp = inp.replace('tb','')
            return int(inp)*1000
        elif ('gb' in inp):
            inp = inp.replace('gb','')
            return int(inp)
        elif ('mb' in inp):
            inp = inp.replace('mb','')
            return float(inp)

In [105]:
def remove_inch (inp):
    if (not isinstance(inp,float)):
        inp = inp.replace('inch','')
        return float(inp)

In [106]:
def find_name (inp):
    test_inp = set(inp.lower().split())
    lenovo = set(["lenovo","legion","ideapad","thinkbook","yoga","loq","thinkpad","لنوو"])
    asus = set(["asus","ایسوس","rog","strix","tuf","vivobook","zenbook","creator"])
    hp = set(["hp","اچ","victus","zbook","pavilion","omen","spectre","اچ‌پی"])
    dell = set(["dell","precision","دل","latitude","inspiron","vostro","xps"])
    acer = set(["acer","ایسر","nitro","aspire","predator"])
    apple = set(["macbook","air","اپل","بوک","ایر"])
    msi = set(["msi","katana","sword","titan"])
    microsoft = set(["microsoft","مایکروسافت","surface","surfacebook"])
    if (lenovo.intersection(test_inp)):
        return "lenovo"
    elif (asus.intersection(test_inp)):
        return "asus"
    elif (hp.intersection(test_inp)):
        return "hp"
    elif (dell.intersection(test_inp)):
        return "dell"
    elif (acer.intersection(test_inp)):
        return "acer"
    elif (apple.intersection(test_inp)):
        return "apple"
    elif (msi.intersection(test_inp)):
        return "msi"
    elif (microsoft.intersection(test_inp)):
        return "microsoft"
    else:
        return "other"

In [107]:
df1 = pd.read_csv("dataset.csv")
df2 = pd.read_csv("prices.csv",index_col=0)
df3 = pd.read_csv("name1.csv",index_col=0)
df4 = pd.read_csv("name2.csv",index_col=0)
df2.index = df3.index
frames = [df1, df2, df3,df4]
df = pd.concat(frames,axis=1)
df.dropna(
    axis=0,
    how='all',
    thresh=None,
    subset=["hdd","ssd"],
    inplace=True
)
df.dropna(
    axis=0,
    how='all',
    thresh=None,
    subset=["name1","name2"],
    inplace=True
)
df['hdd']=df['hdd'].replace(np.nan,"0 gb")
df['ssd']=df['ssd'].replace(np.nan,"0 gb")
df.dropna(axis=0,
        how = 'any',
        thresh = None,
        subset=["cpu","ram","graphic_ram","screen_size"],
        inplace = True
)
df.drop('stock_status',inplace=True,axis=1)
df = df.drop(df[df.prices == 0].index)
df['hdd'] = df['hdd'].map(lambda x : create_number(x))
df['ssd'] = df['ssd'].map(lambda x : create_number(x))
df['screen_size'] = df['screen_size'].map(lambda x : remove_inch(x))
df['ram'] = df['ram'].map(lambda x : create_number(x))
df['is_unified'] = df['graphic_ram'].map(lambda x : np.where('unified' in x,1,0))
df['graphic_ram'] = df['graphic_ram'].map(lambda x : np.where('unified' in x,"0 gb",x))
df['graphic_ram'] = df['graphic_ram'].map(lambda x : create_number(x))
df['company'] = df["name1"].map(lambda x: find_name(x))
df.drop('name1',inplace=True,axis=1)
df.drop('name2',inplace=True,axis=1)
df


,cpu,screen_size,ram,graphic_ram,hdd,ssd,prices,is_unified,company
0,Celeron,15.6,4,0.0,1000,0,8300000,1,lenovo
1,Core i7,15.6,16,8.0,0,512,56599000,0,asus
2,M1,13.0,8,0.0,0,256,37000000,1,apple
3,Core i5,15.6,16,4.0,0,512,34469000,0,lenovo
4,Core i3,15.6,12,0.0,1000,256,15600000,1,lenovo
...,...,...,...,...,...,...,...,...,...
1165,Core i5,15.6,16,4.0,0,512,42492000,0,hp
1166,Core i3,15.6,4,2.0,1000,0,13000000,0,lenovo
1167,Core i5,14.0,16,8.0,0,256,13800000,0,dell
1172,Ryzen 5,15.6,16,4.0,0,1000,38500000,0,lenovo


In [108]:
df.to_csv("D:\works\Term6\ML\project\laptops.csv")